<a href="https://colab.research.google.com/github/Guru-180188/Projects/blob/main/Data_Generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/ .kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d chandhurubaskar/cats-and-dogs

cats-and-dogs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cats-and-dogs.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import pathlib
from torch.utils.data import DataLoader
from torch.optim  import Adam
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
train_path="/content/cat vs dog/train"
test_path="/content/cat vs dog/test"

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)


In [ ]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [ ]:
print(classes)

['cat', 'dog']


In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=6):
        super(ConvNet,self).__init__()

        #Output size after convolution filter
        #((w-f+2P)/s) +1

        #Input shape= (256,3,150,150)

        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)

        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)


        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)



        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)


        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)


    def forward(self,input):
          output=self.conv1(input)
          output=self.bn1(output)
          output=self.relu1(output)

          output=self.pool(output)

          output=self.conv2(output)
          output=self.relu2(output)

          output=self.conv3(output)
          output=self.bn3(output)
          output=self.relu3(output)


              #Above output will be in matrix form, with shape (256,32,75,75)

          output=output.view(-1,32*75*75)


          output=self.fc(output)

          return output




In [ ]:
model=ConvNet(num_classes=2).to(device)
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [ ]:
num_epochs=10

train_count=len(glob.glob(train_path+"/**/*.jpg"))
test_count=len(glob.glob(test_path+"/**/*.jpg"))

In [ ]:
print(train_count)

161


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):

    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0

    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=images.cuda()
            labels=labels.cuda()

        optimizer.zero_grad()

        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()


        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)

        train_accuracy+=int(torch.sum(prediction==labels.data))

    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count


    # Evaluation on testing dataset
    model.eval()

    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=images.cuda()
            labels=labels.cuda()

        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))

    test_accuracy=test_accuracy/test_count


    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))

    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(34.5210) Train Accuracy: 0.45962732919254656 Test Accuracy: 0.1
Epoch: 1 Train Loss: tensor(16.0662) Train Accuracy: 0.5279503105590062 Test Accuracy: 0.7
Epoch: 2 Train Loss: tensor(9.9309) Train Accuracy: 0.6583850931677019 Test Accuracy: 0.8
Epoch: 3 Train Loss: tensor(8.5143) Train Accuracy: 0.7267080745341615 Test Accuracy: 0.4
Epoch: 4 Train Loss: tensor(4.3120) Train Accuracy: 0.7018633540372671 Test Accuracy: 0.9
Epoch: 5 Train Loss: tensor(3.7195) Train Accuracy: 0.7080745341614907 Test Accuracy: 0.4
Epoch: 6 Train Loss: tensor(2.4249) Train Accuracy: 0.7515527950310559 Test Accuracy: 0.9
Epoch: 7 Train Loss: tensor(1.1608) Train Accuracy: 0.8260869565217391 Test Accuracy: 0.7
Epoch: 8 Train Loss: tensor(0.7619) Train Accuracy: 0.8819875776397516 Test Accuracy: 0.9
Epoch: 9 Train Loss: tensor(0.4337) Train Accuracy: 0.8881987577639752 Test Accuracy: 0.3


In [ ]:
checkpoint=torch.load('best_checkpoint.model')
model=ConvNet(num_classes=2)
model.load_state_dict(checkpoint)
model.eval()

ConvNet(
  (conv1): Conv2d(3, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (conv3): Conv2d(20, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (fc): Linear(in_features=180000, out_features=2, bias=True)
)

In [ ]:
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

In [ ]:
def prediction(img_path,transformer):

    image=Image.open(img_path)

    image_tensor=transformer(image).float()


    image_tensor=image_tensor.unsqueeze_(0)

    if torch.cuda.is_available():
        image_tensor.cuda()

    input=image_tensor


    output=model(input)

    index=output.data.numpy().argmax()

    pred=classes[index]

    return pred

In [ ]:
images_path = glob.glob(test_path +'/*.jpg')
print(images_path)
print(test_path)

[]
/content/cat vs dog/test


In [ ]:
pred_dict={}
for i in images_path:

    #pred_dict[i[i.rfind('/')+1:]]=prediction(i,transformer)
    print(i)

In [ ]:
#pred_dict